In [16]:
%load_ext autoreload

In [24]:
%autoreload 2

import torch
import numpy as np

from dataset import SliceDataModule
from common.utils import save_reconstructions
from collections import defaultdict
from tqdm import tqdm
from pathlib import Path

In [25]:
HASH = "aa496a972dd241689c3c0aeb071b759e"
SAVE_DIR = f"reconstructions/{HASH}"

In [26]:
from models import VarNetRoughSensOL

checkpoint = torch.load(f"mlruns/0/{HASH}/artifacts/checkpoints/latest_checkpoint.pth")
model = VarNetRoughSensOL(**checkpoint["hyper_parameters"]["init_args"])
model.load_state_dict(torch.load(f"mlruns/0/{HASH}/artifacts/checkpoints/latest_checkpoint.pth")["state_dict"])
model = model.cuda()

In [27]:
dm = SliceDataModule(root="/home/Data")
dm.setup("test")
dm.setup("predict")

In [28]:
model.eval()
reconstructions = defaultdict(dict)
print("Reconstructing public leaderboard...")
with torch.no_grad():
    for (mask, kspace, target, maximum, fnames, slices) in tqdm(dm.test_dataloader()):
        output = model(kspace.cuda(non_blocking=True), mask.cuda(non_blocking=True))
        output = model.image_space_crop(output)
        for i in range(output.shape[0]):
            reconstructions[fnames[i]][slices[i]] = output[i].cpu().numpy()
    
for fname in reconstructions:
    reconstructions[fname] = np.stack([reconstructions[fname][slice] for slice in sorted(reconstructions[fname])])
print("Saving reconstructions of public leaderboard...")
save_reconstructions(reconstructions, Path(f"{SAVE_DIR}/public"))


model.eval()
reconstructions = defaultdict(dict)
print("Reconstructing private leaderboard...")
with torch.no_grad():
    for (mask, kspace, target, maximum, fnames, slices) in tqdm(dm.predict_dataloader()):
        output = model(kspace.cuda(non_blocking=True), mask.cuda(non_blocking=True))
        output = model.image_space_crop(output)
        for i in range(output.shape[0]):
            reconstructions[fnames[i]][slices[i]] = output[i].cpu().numpy()
    
for fname in reconstructions:
    reconstructions[fname] = np.stack([reconstructions[fname][slice] for slice in sorted(reconstructions[fname])])
print("Saving reconstructions of private leaderboard...")
save_reconstructions(reconstructions, Path(f"{SAVE_DIR}/private"))


Reconstructing public leaderboard...


100%|██████████| 984/984 [04:47<00:00,  3.42it/s]


Saving reconstructions of public leaderboard...
Reconstructing private leaderboard...


100%|██████████| 984/984 [04:43<00:00,  3.48it/s]


Saving reconstructions of private leaderboard...


In [29]:
!sh leaderboard_eval.sh {HASH}

Model: aa496a972dd241689c3c0aeb071b759e

Leaderboard SSIM : 0.9495
========== Details ==========
Leaderboard SSIM (public): 0.9575
Leaderboard SSIM (private): 0.9415
